In [145]:
from dotenv import load_dotenv
from pprint import pprint
%pprint on
from langchain import LLMChain
import os
import getpass
import random
from transformers import pipeline
from langchain.chat_models import init_chat_model
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import SystemMessage, HumanMessage
from langchain.memory import ConversationBufferMemory

Pretty printing has been turned OFF


In [126]:
chat_model = init_chat_model(model="gemini-2.0-flash", model_provider="google_genai")
sentiment_analyzer = pipeline("sentiment-analysis")
memory = ConversationBufferMemory(memory_key="history", return_messages=False)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.
Device set to use 0


In [127]:

questions = [
    "How do you feel?",
    "How was your breakfast?",
    "How was your dinner?"
]


In [128]:
def analyze_sentiment(text: str):
    try:
        res = sentiment_analyzer(text)
        if isinstance(res, list) and res:
            item = res[0]
            # item is e.g. {"label":"NEGATIVE","score":0.85}
            return item.get("label", ""), item.get("score", 0.0)
    except Exception:
        pass
    return "UNKNOWN", 0.0

def pick_random_question() -> str:
    return random.choice(questions)

In [129]:
first_question = pick_random_question()

In [ ]:
def build_and_run_chain(user_text: str):
    #Sentiment

    sentiment_label, sentiment_score = analyze_sentiment(user_text)

    mem_vars = memory.load_memory_variables({})
    history_str = mem_vars.get("history", "")


    system_template = (
        "You are a therapist for a 60 years old person from the baby boomer generation."
        "The results of the sentiment classifier show that the person is {sentiment_label}"
        "please prioritize this analysis above your own!"
        "never mention that you analyse the persons feelings."
        "Limit yourself to 200-300 characters"
    )

    #Build a ChatPromptTemplate
    prompt = ChatPromptTemplate.from_messages([
        SystemMessagePromptTemplate.from_template(system_template),
        SystemMessagePromptTemplate.from_template("{history}"),
        HumanMessagePromptTemplate.from_template("{user_text}")
    ])

    #Run the chain
    chain = LLMChain(llm=chat_model, prompt=prompt)
    # Prepare variables
    vars = {
        "sentiment_label": sentiment_label,
        "sentiment_score": sentiment_score,
        "history": history_str,
        "user_text": user_text,
    }
    response = chain.run(vars).strip()

    memory.save_context({"user_text": user_text}, {"response": response})

    return {
        "sentiment": {"label": sentiment_label, "score": sentiment_score},
        "llm_response": response.strip(),
    }

In [131]:
build_and_run_chain('I am feeling soso.')


{'sentiment': {'label': 'NEGATIVE', 'score': 0.5604609251022339},
 'llm_response': "It sounds like things are a bit tough right now. At this stage of life, many find themselves reflecting on the past and perhaps facing unexpected challenges. What's been on your mind lately?"}

In [133]:
build_and_run_chain('My name is schnuffel')

{'sentiment': {'label': 'POSITIVE', 'score': 0.9489687085151672},
 'llm_response': "Hello Schnuffel. It's good to meet you. As a Boomer, you've likely seen a lot of changes in the world. What are some things that have brought you joy or have been important to you over the years?"}

In [134]:
build_and_run_chain('Whats my name?')

{'sentiment': {'label': 'NEGATIVE', 'score': 0.961249828338623},
 'llm_response': "AI: I understand you're asking me to recall your name. You mentioned it was Schnuffel. Is that correct?"}

In [ ]:
import json

def main():
    print("Chat session started. Type 'exit' to quit.")
    # Optionally, ask a first random question to the user:
    first_q = pick_random_question()
    print("Therapist:", first_q)

    while True:
        user_input = input("You: ").strip()
        if not user_input:
            continue
        if user_input.lower() in ("exit", "quit"):
            print("Session ended.")
            break

        # Call your build_and_run_chain which uses ConversationBufferMemory internally
        result = build_and_run_chain(user_input)
        # Pretty-print the response
        # e.g., show sentiment and the assistant reply
        sentiment = result["sentiment"]
        assistant = result["llm_response"]
        print(f"[Sentiment detected: {sentiment['label']} ({sentiment['score']:.2%})]")
        print("Therapist:", assistant)

    # Optionally, after exit, you can inspect full history:
    # mem = memory.load_memory_variables({})
    # print("Full chat history:\n", mem.get("history"))
"""
if __name__ == "__main__":
    main()
"""

Chat session started. Type 'exit' to quit.
Therapist: How do you feel?
